# TensorFlow crash course
### **PART 4**

### Previous dataset(MNIST) loading

In [1]:
import numpy as np 
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.utils import to_categorical 

(X_train, y_train), (X_test, y_test) = mnist.load_data() 
X_train, X_test = X_train[:10000], X_test[:1000]
y_train, y_test = y_train[:10000], y_test[:1000]
X_train, X_test = X_train.reshape(-1, 28*28) / 255.0, X_test.reshape(-1, 28*28) / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(10000, 784) (1000, 784) (10000, 10) (1000, 10)


### Custom metric
#### Some metrics are called **Streaming Metrics** due to being updated gradually, batch after batch like precision

In [2]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Metric
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential, load_model


#  Implement a simple ratio metric(y_true over y_pred)
class Ratio(Metric):
    def __init__(self, **kwargs): 
         super().__init__(**kwargs)
         #  add_weight() method creates the variables needed to keep track of
         self.mean_ratio = self.add_weight("mean_ratio", initializer="zeros")

    #  Updater
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.add(y_true, tf.constant(1, dtype=tf.float32))
        y_pred = tf.add(y_pred, tf.constant(1, dtype=tf.float32))
        ratio = tf.divide(y_true, y_pred)
        ratio = tf.reduce_mean(ratio)
        self.mean_ratio.assign(ratio)

    def result(self):
        return self.mean_ratio
    
    #  Reset the variables
    def reset_state(self):
        self.mean_ratio.assign(0)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config}

dnn = Sequential()
dnn.add(Dense(128, input_shape=(784, ), activation="relu"))
dnn.add(Dense(64, activation="relu"))
dnn.add(Dense(32, activation="relu"))
dnn.add(Dense(10, activation="softmax"))
dnn.compile(loss="categorical_crossentropy", optimizer=SGD(learning_rate=0.01), metrics=[Ratio()])
dnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
dnn.save("custom_component_model.h5") 
dnn = load_model("custom_component_model.h5", custom_objects={"Ratio":Ratio})
dnn.evaluate(X_test, y_test, batch_size=32)

Epoch 1/10
313/313 [==============================] - 2s 3ms/step - loss: 1.7054 - ratio: 0.9908 - val_loss: 1.0604 - val_ratio: 0.9972
Epoch 2/10
313/313 [==============================] - 1s 2ms/step - loss: 0.6838 - ratio: 0.9975 - val_loss: 0.6153 - val_ratio: 1.0014
Epoch 3/10
313/313 [==============================] - 1s 2ms/step - loss: 0.4619 - ratio: 0.9961 - val_loss: 0.5123 - val_ratio: 1.0037
Epoch 4/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3787 - ratio: 0.9967 - val_loss: 0.4704 - val_ratio: 1.0011
Epoch 5/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3311 - ratio: 0.9967 - val_loss: 0.3927 - val_ratio: 0.9983
Epoch 6/10
313/313 [==============================] - 1s 2ms/step - loss: 0.2978 - ratio: 1.0000 - val_loss: 0.3750 - val_ratio: 0.9993
Epoch 7/10
313/313 [==============================] - 1s 2ms/step - loss: 0.2724 - ratio: 0.9982 - val_loss: 0.3563 - val_ratio: 0.9981
Epoch 8/10
313/313 [============================

[0.2973668873310089, 0.9974361658096313]

### Custom layer
#### A simple dense layer

In [3]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.activations import get, serialize
from tensorflow.keras.models import Sequential, load_model

class denseLayer(Layer):
    def __init__(self, nodes, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.nodes = nodes 
        #  Convert the given activation to a appropriate activation function 
        self.activation = get(activation)

    #  Create the layer variables and get the previous input shape
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.nodes],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.nodes], initializer="zeros")
        super().build(batch_input_shape)

    #  Compute the desired operations
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    #  Return the output shape of your custom layer shape
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.nodes])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "nodes": self.nodes,
            "activation": serialize(self.activation)}

dnn = Sequential()
dnn.add(Dense(128, input_shape=(784, ), activation="relu"))
dnn.add(denseLayer(64, activation="relu"))
dnn.add(denseLayer(32, activation="relu"))
dnn.add(denseLayer(10, activation="softmax"))
dnn.compile(loss="categorical_crossentropy", optimizer=SGD(learning_rate=0.01), metrics=[Ratio()])
dnn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
dnn.save("custom_component_model.h5") 
dnn = load_model("custom_component_model.h5", custom_objects={"Ratio":Ratio, "denseLayer":denseLayer})
dnn.evaluate(X_test, y_test, batch_size=32)

Epoch 1/10
313/313 [==============================] - 1s 3ms/step - loss: 1.7447 - ratio_1: 0.9921 - val_loss: 1.1528 - val_ratio_1: 0.9982
Epoch 2/10
313/313 [==============================] - 1s 2ms/step - loss: 0.7430 - ratio_1: 0.9957 - val_loss: 0.6146 - val_ratio_1: 1.0002
Epoch 3/10
313/313 [==============================] - 1s 2ms/step - loss: 0.4713 - ratio_1: 0.9987 - val_loss: 0.4793 - val_ratio_1: 1.0031
Epoch 4/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3806 - ratio_1: 0.9956 - val_loss: 0.4119 - val_ratio_1: 1.0038
Epoch 5/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3339 - ratio_1: 1.0008 - val_loss: 0.3985 - val_ratio_1: 1.0025
Epoch 6/10
313/313 [==============================] - 1s 2ms/step - loss: 0.3006 - ratio_1: 0.9976 - val_loss: 0.3557 - val_ratio_1: 1.0027
Epoch 7/10
313/313 [==============================] - 1s 2ms/step - loss: 0.2763 - ratio_1: 1.0005 - val_loss: 0.3465 - val_ratio_1: 1.0056
Epoch 8/10
313/313 [

[0.32572755217552185, 1.0049924850463867]

## Bonus
### Multi-Input layer

In [4]:
from tensorflow.keras.layers import Layer
class MyMultiLayer(Layer):
    def call(self, X):
        X1, X2 = X
        return [X1, X2, X1 + X2]
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input
        return [b1, b2, b1]